In [1]:
import pandas as pd
import numpy as np
import gc
import torch
from random import sample

gc.enable()
gc.collect()

0

In [4]:
logs_0708 = pd.read_parquet('./data/logs_df_2024-07-08.parquet')
df_stats = pd.read_parquet('./data/video_stat.parquet')

In [5]:
logs = pd.merge(logs_0708, df_stats[['video_id','v_duration']],on='video_id', how='left')

In [6]:
logs['is_long_watch'] = logs.apply(
    lambda row: 1 if (
        (row['v_duration'] > 300 and row['watchtime'] / row['v_duration'] > 0.25) or
        (row['v_duration'] <= 300 and row['watchtime'] > 30) or
        (row['v_duration'] <= 30 and (row['watchtime'] != 0 and row['v_duration'] != 0 and row['watchtime'] / row['v_duration'] > 0.75))
    ) else 0,
    axis=1
)

In [7]:
watchtime_sum_by_user = logs.groupby('user_id')['watchtime'].sum().reset_index()

In [8]:
watchtime_sum_by_user = watchtime_sum_by_user.set_index('user_id')

In [9]:
users_to_drop = watchtime_sum_by_user.loc[(watchtime_sum_by_user.watchtime > 43200) | (watchtime_sum_by_user.watchtime <=2.5)]

In [10]:
filtered_df = logs[~logs['user_id'].isin(users_to_drop)]

In [11]:
logs = filtered_df.iloc[filtered_df.groupby(by=['user_id', 'video_id'])['watchtime'].idxmax()]

In [12]:
logs = logs.drop(columns=[i for i in logs.columns if i not in ['user_id', 'video_id', 'is_long_watch']])

In [13]:
user_counts = logs['user_id'].value_counts()

In [14]:
filtered_users = user_counts[(user_counts >= 15) & (user_counts <= 70)]

In [15]:
filtered_users_sorted = filtered_users.sort_values(ascending=False)

In [48]:
logs_tmp = logs[logs['user_id'].isin(filtered_users_sorted.index.values)]

In [49]:
logs_tmp = logs_tmp.iloc[sample(range(0, len(logs_tmp)), 50000)]

In [50]:
logs_tmp

,user_id,video_id,is_long_watch
9529991,9b08f2c8-7525-42d7-94f2-e3509f93c00b,223a8973-b718-4336-9d4e-88ee4ee79fed,1
15818495,f86781a5-451e-4f28-a408-30c0f6869377,1402da87-e18a-4eb6-afc2-bbb23359205a,0
3670108,0c673a92-7e17-44af-84fd-fbaf283a0357,c9a3eeb0-e084-4299-9f08-675136b40976,1
1781165,aa4674c0-4bd9-4a55-a722-52caffe9a5b6,b207d280-6c4e-4f46-a176-e475234dcf73,0
9590425,62bfff8d-22e2-44aa-a73d-d40ccb713469,b074abc1-3a9c-454f-bcc6-d0762466557e,1
...,...,...,...
12242580,60e8c419-e4c8-44f0-a81d-d71e4b977302,5d17920f-2b1b-4565-b1d7-5ca08564b503,1
3642540,544813f7-14e7-4ad2-a47d-58b5ebf612cd,8206b608-bd92-4126-bb15-1fb1b8db5614,1
2059537,de588ea6-aa8e-45a9-8eec-349cfde84650,0da49171-e177-494f-b1a2-8d190b3316b6,0
7216114,639b4088-9c9c-4d08-bbcc-fa0e7dfa34e1,826ce756-8c38-4194-a96b-02aa93bf6bca,1


In [51]:
filtered_df_stats = df_stats[df_stats['video_id'].isin(logs_tmp['video_id'])]

In [53]:
filtered_df_stats.drop(columns = ['row_number', 'title', 'description', 'category_id', 'author_id', 'v_pub_datetime'], inplace=True)

/tmp/ipykernel_14463/2830144736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_stats.drop(columns = ['row_number', 'title', 'description', 'category_id', 'author_id', 'v_pub_datetime'], inplace=True)


In [55]:
filtered_df_stats.rename(columns = {'video_id' : 'item_id'}, inplace=True)
filtered_df_stats

/tmp/ipykernel_14463/2564048367.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_stats.rename(columns = {'video_id' : 'item_id'}, inplace=True)


,item_id,v_total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,v_duration,v_cr_click_like_7_days,...,v_avg_watchtime_7_day,v_avg_watchtime_30_day,v_frac_avg_watchtime_1_day_duration,v_frac_avg_watchtime_7_day_duration,v_frac_avg_watchtime_30_day_duration,v_category_popularity_percent_7_days,v_category_popularity_percent_30_days,v_long_views_1_days,v_long_views_7_days,v_long_views_30_days
3,154fac0e-1794-4406-afa0-61f688e3a764,0,10657,5227,1300,202,24,0,42.025,0.000000,...,97.190000,122.290606,1.370118,2.312671,2.909949,0.418336,0.513562,186,1162,4550
38,2de9e412-e15d-4819-9d42-567342e87acd,0,8670,3655,1013,147,21,0,1352.289,0.296150,...,817.213228,819.243228,0.598401,0.604318,0.605820,9.304933,8.716591,99,692,2482
40,d113f99c-1001-4775-9ee1-8b4ab4cc45f4,3,118455,43554,9298,1284,22,3,124.761,0.010755,...,95.016670,91.678032,0.923829,0.761590,0.734829,6.106227,5.319095,0,5973,31299
43,585cee67-ff42-4c24-9da7-17eec76b6bdc,13,39687,11180,2261,291,21,3,25.600,0.044228,...,14.968598,13.518157,0.562983,0.584711,0.528053,2.221145,2.315903,0,3,17
45,dc7ccfb8-b8ed-4732-8c1d-be69dfde31d7,0,129224,24045,5882,845,10,2,35.076,0.000000,...,20.370452,23.672822,0.535068,0.580752,0.674901,5.017427,4.373750,0,2971,13235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935848,ff1871a2-1cab-426c-8d35-5238f36ce77f,16,48241,4143,999,154,72,5,2453.200,0.400400,...,1439.807808,1389.790249,0.628384,0.586910,0.566521,19.091831,20.732010,107,672,2742
1935959,4fc069be-d88e-403c-b1dc-18d4ee3314d8,4,11838,4896,1616,304,19,1,879.967,0.185644,...,289.280322,292.837010,0.345557,0.328740,0.332782,8.097824,13.016936,0,466,1759
1935965,4d0952b9-97c0-4828-b032-3f9b07cf4577,1,5054,3367,2283,382,6,2,10.167,0.000000,...,2.724047,3.782002,0.220146,0.267930,0.371988,2.054214,1.749159,0,16,38
1935978,d490a7bf-64ac-4cfb-b9fb-5d3e84545710,0,458,103,35,11,0,0,1223.112,0.000000,...,501.285714,405.174757,0.460078,0.409844,0.331265,4.858309,5.721517,0,11,37


In [63]:
filtered_df_stats.to_csv('./stats_sample50k.csv')

In [58]:
logs_tmp.rename(columns = {'video_id' : 'item_id'}, inplace=True)

In [64]:
logs_tmp.to_csv('./ui_sample50k.csv')

In [62]:
!ls data

logs_df_2024-07-08.parquet  logs_df_2024-07-24.parquet
logs_df_2024-07-09.parquet  logs_df_2024-07-25.parquet
logs_df_2024-07-10.parquet  logs_df_2024-07-26.parquet
logs_df_2024-07-11.parquet  logs_df_2024-07-27.parquet
logs_df_2024-07-12.parquet  logs_df_2024-07-28.parquet
logs_df_2024-07-13.parquet  logs_df_2024-07-29.parquet
logs_df_2024-07-14.parquet  logs_df_2024-07-30.parquet
logs_df_2024-07-15.parquet  logs_df_2024-07-31.parquet
logs_df_2024-07-16.parquet  logs_df_2024-08-01.parquet
logs_df_2024-07-17.parquet  logs_df_2024-08-02.parquet
logs_df_2024-07-18.parquet  logs_df_2024-08-03.parquet
logs_df_2024-07-19.parquet  logs_df_2024-08-04.parquet
logs_df_2024-07-20.parquet  logs_df_2024-08-05.parquet
logs_df_2024-07-21.parquet  logs_df_2024-08-06.parquet
logs_df_2024-07-22.parquet  video_stat.parquet
logs_df_2024-07-23.parquet


In [2]:
import os

In [7]:
def read_logs(file):

    logs_0708 = pd.read_parquet(file)
    df_stats = pd.read_parquet('./data/video_stat.parquet')
    
    logs = pd.merge(logs_0708, df_stats[['video_id','v_duration']],on='video_id', how='left')
    
    logs['is_long_watch'] = logs.apply(
        lambda row: 1 if (
            (row['v_duration'] > 300 and row['watchtime'] / row['v_duration'] > 0.25) or
            (row['v_duration'] <= 300 and row['watchtime'] > 30) or
            (row['v_duration'] <= 30 and (row['watchtime'] != 0 and row['v_duration'] != 0 and row['watchtime'] / row['v_duration'] > 0.75))
        ) else 0,
        axis=1
    )
    
    watchtime_sum_by_user = logs.groupby('user_id')['watchtime'].sum().reset_index()
    watchtime_sum_by_user = watchtime_sum_by_user.set_index('user_id')
    
    #users_to_drop = watchtime_sum_by_user.loc[(watchtime_sum_by_user.watchtime > 43200) | (watchtime_sum_by_user.watchtime <=2.5)]
    
    #filtered_df = logs[~logs['user_id'].isin(users_to_drop)]
    
    logs = watchtime_sum_by_user.iloc[watchtime_sum_by_user.groupby(by=['user_id', 'video_id'])['watchtime'].idxmax()]
    
    logs = logs.drop(columns=[i for i in logs.columns if i not in ['user_id', 'video_id', 'is_long_watch']])
    
    user_counts = logs['user_id'].value_counts()
    
    filtered_users = user_counts[(user_counts >= 15) & (user_counts <= 70)]
    
    filtered_users_sorted = filtered_users.sort_values(ascending=False)
    
    logs_tmp = logs[logs['user_id'].isin(filtered_users_sorted.index.values)]
    
    logs_tmp = logs_tmp.iloc[sample(range(0, len(logs_tmp)), 5000)]
    
    print(file, '  is Done')
    
    return logs_tmp

In [ ]:
folder_path = './data'

parquet_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.startswith('logs') and file.endswith('.parquet')]

dfs = [read_logs(file) for file in parquet_files]

concatenated_df = pd.concat(dfs, ignore_index=True)

In [ ]:
filtered_df_stats = df_stats[df_stats['video_id'].isin(concatenated_df['video_id'])]
filtered_df_stats.drop(columns = ['row_number', 'title', 'description', 'category_id', 'author_id', 'v_pub_datetime'], inplace=True)
filtered_df_stats.rename(columns = {'video_id' : 'item_id'}, inplace=True)

In [ ]:
filtered_df_stats.to_csv('./stats_allsample5k.csv')

In [ ]:
logs_tmp.to_csv('./ui_allsample5k.csv')